<a href="https://colab.research.google.com/github/vidhya2324/Neutral_network/blob/main/Movie_review_Sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LSTM is comes under the RNN

In [1]:
!pip install kaggle

**Importing the dependencies**

In [3]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Data collection**

In [18]:
import json
kaggle_dictionary=json.load(open("kaggle.json"))

In [19]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [20]:
#setup AIP credential
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [25]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 74% 19.0M/25.7M [00:00<00:00, 199MB/s]
100% 25.7M/25.7M [00:00<00:00, 223MB/s]


In [26]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [27]:
#unzip the file
!unzip \*.zip && rm *.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [28]:
!ls

'IMDB Dataset.csv'   kaggle.json   sample_data


**Loading the dataset and preprocession**

In [29]:
Movie_reviews=pd.read_csv("/content/IMDB Dataset.csv")

In [31]:
Movie_reviews.shape

(50000, 2)

In [33]:
Movie_reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [35]:
Movie_reviews['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [36]:
Movie_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [37]:
#replace the object into number
Movie_reviews.replace({'sentiment':{'positive':1,'negative':0}},inplace=True)

<ipython-input-37-e3c1572f4c66>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Movie_reviews.replace({'sentiment':{'positive':1,'negative':0}},inplace=True)


In [38]:
Movie_reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [39]:
Movie_reviews['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


In [40]:
#split the data into train and test data
train_data,test_data=train_test_split(Movie_reviews,test_size=0.2,random_state=42)

In [41]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


pad_sequences is used to allows batches of text data with different sequence lengths to be processed together efficiently.
together all the input into same length.

In [44]:
#Data preprocessing(its a important phase in NLP model)
#tokenizer text data
tokenizer=Tokenizer(num_words=5000)   #tokenizer is used to convert the words into random numbers
tokenizer.fit_on_texts(train_data['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [45]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [47]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [48]:
y_train=train_data['sentiment']
y_test=test_data['sentiment']

#Building the Neutral network in LSTM


Its very used for time series dataset and the data in sequential, then we use lstm rnn. because these time series is depend on the previous output

In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))  # Embedding layer with input shape
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))               # LSTM layer
model.add(Dense(1, activation="sigmoid"))                              # Output layer

model.build(input_shape=(None, 200))  # Explicitly build the model with input shape

model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [62]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [64]:
#Training the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 141s 267ms/step - accuracy: 0.7208 - loss: 0.5371 - val_accuracy: 0.8081 - val_loss: 0.4273
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 133s 260ms/step - accuracy: 0.8369 - loss: 0.3767 - val_accuracy: 0.8659 - val_loss: 0.3244
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 129s 257ms/step - accuracy: 0.8785 - loss: 0.3069 - val_accuracy: 0.8394 - val_loss: 0.3643
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 141s 255ms/step - accuracy: 0.8900 - loss: 0.2810 - val_accuracy: 0.8700 - val_loss: 0.3212
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 143s 256ms/step - accuracy: 0.9055 - loss: 0.2414 - val_accuracy: 0.8719 - val_loss: 0.3191


In [67]:
#Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 86ms/step - accuracy: 0.8775 - loss: 0.3110
Test Loss: 0.30870354175567627
Test Accuracy: 0.8770999908447266


**Building the prediction in real time**

In [68]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [69]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
The sentiment of the review is: positive


In [70]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
The sentiment of the review is: positive
